In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.307752  [    0/60000]
loss: 2.297606  [ 6400/60000]
loss: 2.277105  [12800/60000]
loss: 2.274512  [19200/60000]
loss: 2.245250  [25600/60000]
loss: 2.217127  [32000/60000]
loss: 2.234683  [38400/60000]
loss: 2.198328  [44800/60000]
loss: 2.196704  [51200/60000]
loss: 2.159643  [57600/60000]
Test Error: 
 Accuracy: 34.5%, Avg loss: 2.158396 

Epoch 2
-------------------------------
loss: 2.171895  [    0/60000]
loss: 2.163241  [ 6400/60000]
loss: 2.109985  [12800/60000]
loss: 2.127656  [19200/60000]
loss: 2.076336  [25600/60000]
loss: 2.010983  [32000/60000]
loss: 2.050730  [38400/60000]
loss: 1.969948  [44800/60000]
loss: 1.971656  [51200/60000]
loss: 1.907047  [57600/60000]
Test Error: 
 Accuracy: 52.4%, Avg loss: 1.901654 

Epoch 3
-------------------------------
loss: 1.935804  [    0/60000]
loss: 1.909934  [ 6400/60000]
loss: 1.795636  [12800/60000]
loss: 1.836122  [19200/60000]
loss: 1.731772  [25600/60000]
loss: 1.668344  [32000/600